In [1]:
pip install fosforml

     |████████████████████████████████| 40kB 4.0MB/s eta 0:00:011
     |████████████████████████████████| 10.9MB 10.9MB/s eta 0:00:01██▊                  | 4.7MB 10.9MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 94.9MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 98.7MB/s eta 0:00:01
     |████████████████████████████████| 307kB 76.3MB/s eta 0:00:01
     |████████████████████████████████| 174kB 97.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 30.4MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 365kB/s eta 0:00:011
     |████████████████████████████████| 1.3MB 69.9MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 74kB/s  eta 0:00:011
     |████████████████████████████████| 61kB 25.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 18.8MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 75.4MB/s eta 0:00:01
     |████████████████████████████████| 133kB 106.2MB/s eta 0:00:01
     |█████

In [7]:
pip install seaborn

     |████████████████████████████████| 296kB 7.2MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
!pip install pandas
!pip install snowflake-ml-python
!pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
pip install pandas-profiling

     |████████████████████████████████| 327kB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 358kB 14.4MB/s eta 0:00:01
     |████████████████████████████████| 296kB 16.3MB/s eta 0:00:01
     |████████████████████████████████| 696kB 29.2MB/s eta 0:00:01
     |████████████████████████████████| 112kB 42.8MB/s eta 0:00:01
     |████████████████████████████████| 522kB 42.8MB/s eta 0:00:01
     |████████████████████████████████| 440kB 41.0MB/s eta 0:00:01
     |████████████████████████████████| 10.8MB 357kB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 97.2MB/s eta 0:00:01
     |████████████████████████████████| 4.5MB 83.9MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 87.8MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 71.1MB/s eta 0:00:01
     |████████████████████████████████| 235kB 91.7MB/s eta 0:00:01
     |████████████████████████████████| 43.9MB 511kB/s eta 0:00:011
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp39-none

In [2]:
import pandas as pd
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [12]:
import seaborn as sns

In [13]:
my_session.connection.database

'ASSORTMENT_PLANNING'

In [4]:
my_session.connection.schema

'CPG_BRONZE'

In [14]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [27]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [28]:
type(sf_df)

snowflake.snowpark.dataframe.DataFrame

In [30]:
sf_df.show()

SnowparkSQLException: (1304): 01b73fd7-0002-f527-0000-31ef000ca046: 090073 (22000): Warehouse 'FDC_WAREHOUSE' cannot be resumed because resource monitor 'FDC_WAREHOUSE_RM' has exceeded its quota.